In [1]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim

import numpy as np
import sklearn.metrics
import pandas as pd
import random

import dataset
import data_reader
import plots

In [2]:
# Choosing device for tensor processing

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using GPU :", device)
else:
    device = torch.device("cpu")
    print("Using CPU ")

Using GPU : cuda:0


In [3]:
EXP_NAME = "tcia_25ep" #"25ep0"#

DATA_SET_NAME = "tcia_data_set_SPLIT" #f"data_set_X20_100%_SPLIT"#

N_PATCHES = 400 # Number of patches to take from each WSI

EPOCHS = 10

In [4]:
def init_weights(m): # XAVIER initialization for final layer weight initialization
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

def init():
    global model, loss_function, optimizer

    # Final dense layer classifier for WSI
    model = nn.Sequential(
        nn.Linear(N_PATCHES, 2),
        #nn.ReLU(inplace=True),
        #nn.Linear(256,2),
        nn.Softmax(dim=-1)
    ).to(device)

    for param in model.parameters():
        param.requires_grad = True

    model.apply(init_weights) # Xavier init
    
    #Hyperparameters:
    learning_rate = 1E-3 # LR
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)#, betas=(0.9, 0.999), eps=1e-08)# Optimizer

    loss_function = nn.BCELoss()  # Loss

In [5]:
def patch_feature_extraction(wsi_data_set):
    global net
    """
    Input: 
    - wsi_data_set, patches from a given WSI, dataset object
    - y, patch labels
    Outputs:
    - output: features of the selected N_PATCHES patches
    """
    wsi_dataloader = DataLoader(wsi_data_set, batch_size=1)
    outputs = []

    for batch_X, batch_y in iter(wsi_dataloader):
        batch_X, batch_y = batch_X.type(torch.FloatTensor).to(device).permute(0, 3, 2, 1), batch_y.type(torch.FloatTensor).to(device) 

        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        normalize = transforms.Normalize(mean=mean, std=std)

        for i, x in enumerate(batch_X):
            batch_X[i] = normalize(batch_X[i]/255) # NP array
    
        output = net(batch_X)[0]
        outputs.append(output)

    outputs = torch.tensor(outputs).to(device)
    
    return outputs

In [6]:
def fwd_pass(wsi_data_set, train=False):

    X = torch.tensor(wsi_data_set.inputs).to(device)
    y = torch.tensor(wsi_data_set.labels[0][0],dtype=torch.float).to(device)

    if train: 
        model.zero_grad()

    with torch.no_grad():
        features = patch_feature_extraction(wsi_data_set)

    output = model(features)
    #print(output[0], y)
    
    y_pred = [torch.argmax(output)]
    y_true = [torch.argmax(y)]

    loss = loss_function(output[0], y)
    #print(loss)
    #loss.requires_grad = True

    if train:
        loss.backward() # Calculate gradients using backprop
        optimizer.step() # Updates W and b using previously calculated gradients

    return [loss, y_pred, y_true]

In [7]:
n_splits = 10
for SPLIT in range(n_splits):
    
    X, y, case_ids = [], [], []

    init() # Initializing patch merging model
        
    MODEL_NAME = EXP_NAME + f"{SPLIT}"

    TRAIN_SPLITS = list(range(n_splits))
    # We take out the SPLIT and SPLIT+1 sets for val and testing
    if SPLIT == n_splits-1: # For the final split for validation we take the first one for test
        TRAIN_SPLITS.remove(0) 
    else:
        TRAIN_SPLITS.remove(SPLIT+1)
    TRAIN_SPLITS.remove(SPLIT)
    
    if SPLIT == n_splits-1: # For the final split for validation we take the firs one for test
        SPLIT_NAME = DATA_SET_NAME + f"{0}" # Test set 
    else:
        SPLIT_NAME = DATA_SET_NAME + f"{SPLIT+1}" # Test set
    
    MODEL_NAME = EXP_NAME + f"{SPLIT}"

    print(SPLIT_NAME)

    net = torch.load(f"C:\\Users\\Alejandro\\Desktop\\heterogeneous-data\\results\\WSI\\models\\{MODEL_NAME}.pth") # Model loading
    net.eval()
    net.fc = nn.Sequential(nn.AvgPool1d(kernel_size=512))

    # Loading training splits:
    for i in TRAIN_SPLITS:
        print(f"Loading training split: {i}")
        SPLIT_NAME = DATA_SET_NAME + f"{i}"
        
        X_, y_, _ , patch_ids_ = data_reader.read_lmdb(f"D:/data/WSI/patches/{SPLIT_NAME}")

        X.extend(X_)
        y.extend(y_)
        case_ids.extend(patch_ids_)

    # Loading validation splits:
    SPLIT_NAME = DATA_SET_NAME + f"{SPLIT}"

    print(f"Loading validation split: {SPLIT}")

    val_X, val_y, _ , val_case_ids = data_reader.read_lmdb(f"D:/data/WSI/patches/{SPLIT_NAME}")


    sample_ids = [case_id.split("_")[1] for case_id in case_ids] # Taking only the sample_id, not patch_id
    val_sample_ids = [case_id.split("_")[1] for case_id in val_case_ids]

    unique_sample_ids = np.unique(sample_ids)
    val_unique_sample_ids = np.unique(sample_ids)
    random.shuffle(unique_sample_ids)
    random.shuffle(val_unique_sample_ids)
    sample_ids = np.array(sample_ids)
    val_sample_ids = np.array(val_sample_ids)

    for EPOCH in range(EPOCHS):
        print("EPOCH: ", EPOCH+1)
        losses, outputs, labels = [], [], []
        val_losses, val_outputs, val_labels = [], [], []

        for unique_sample_id in tqdm(unique_sample_ids):

            ii = np.where(sample_ids == unique_sample_id)[0]

            wsi_data_set = dataset.PatchDataset([], [], [])
            feature_data_set = dataset.PatchDataset([], [], [])

            wsi_data_set.inputs.extend(X[ii[0]:ii[-1]][:N_PATCHES])# Taking the patches from a given wsi
            wsi_data_set.labels.extend(y[ii[0]:ii[-1]][:N_PATCHES])
            wsi_data_set.case_ids.extend(sample_ids[:N_PATCHES])

            if len(wsi_data_set) < N_PATCHES:
                print("No patches")
            else:
                model.train()
                output = fwd_pass(wsi_data_set, train=True)
                losses.append(float(output[0]))
                outputs.append(output[1][0].cpu())
                labels.append(output[2][0].cpu())

            del wsi_data_set
        
        for unique_sample_id in tqdm(val_unique_sample_ids):

            ii = np.where(sample_ids == unique_sample_id)[0]

            wsi_data_set = dataset.PatchDataset([], [], [])
            feature_data_set = dataset.PatchDataset([], [], [])

            wsi_data_set.inputs.extend(X[ii[0]:ii[-1]][:N_PATCHES])# Taking the patches from a given wsi
            wsi_data_set.labels.extend(y[ii[0]:ii[-1]][:N_PATCHES])
            wsi_data_set.case_ids.extend(sample_ids[:N_PATCHES])

            if len(wsi_data_set) < N_PATCHES:
                print("No patches")
            else:
                model.train()
                output = fwd_pass(wsi_data_set, train=False)
                val_losses.append(float(output[0]))
                val_outputs.append(output[1][0].cpu())
                val_labels.append(output[2][0].cpu())

            del wsi_data_set
                
        train_loss = np.mean(losses)
        train_acc = sklearn.metrics.accuracy_score(labels, outputs)
        train_bacc = sklearn.metrics.balanced_accuracy_score(labels,outputs)

        val_loss = np.mean(losses)
        val_acc = sklearn.metrics.accuracy_score(labels, outputs)
        val_bacc = sklearn.metrics.balanced_accuracy_score(labels,outputs)

        print("Train Loss: ", train_loss, "Train ACC:", train_acc, "Train BACC", train_bacc)

        print("Val Loss: ", val_loss, "Val ACC:", val_acc, "Val BACC", val_bacc)

tcia_data_set_SPLIT1
Loading training split: 2
Read lmdb
20366
Loading training split: 3
Read lmdb
20818
Loading training split: 4
Read lmdb
20947
Loading training split: 5
Read lmdb
26374
Loading training split: 6
Read lmdb
14995
Loading training split: 7
Read lmdb
10407
Loading training split: 8
Read lmdb
22786
Loading training split: 9
Read lmdb
19800
EPOCH:  1


  0%|          | 0/439 [00:00<?, ?it/s]

C:\Users\Alejandro\AppData\Local\Temp\ipykernel_3272\1371018220.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  X = torch.tensor(wsi_data_set.inputs).to(device)
c:\Users\Alejandro\anaconda3\envs\openslide\lib\site-packages\torch\utils\data\_utils\collate.py:146: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_numpy.cpp:178.)
  return default_collate([torch.as_tensor(

No patches
No patches
No patches
No patches
No patches
No patches
No patches
No patches
No patches
